In [5]:
# will autoupdate any of the packages imported:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
import pycliffordv2 as pc

In [7]:
bond_len = 3.8
molecule = [["H", [0, 0.0, 0]], ["Li", [bond_len, 0, 0]]]
pyc_h, E0, shift = pc.qchem.qchem_hamiltonian(molecule, use_pyscf=False,multiplicity=1)

Done calculating qubit_op.
Done calculating pyclifford Hamiltonian.
Using exact ground state


In [8]:
pyc_h

-0.24 IIIIIIIZ +0.08 IIIIIIIX +0.04 IIIIIIZI -0.31 IIIIIIZZ -0.08 IIIIIIZX -0.06 IIIIIZII +0.05 IIIIIZIZ +0.01 IIIIIZIX -0.43 IIIIIZZI +0.04 IIIIIZZZ -0.01 IIIIIZZX +0.41 IIIIZIII -0.05 IIIIZIIZ +0.01 IIIIZIIX +0.07 IIIIZIZI -0.05 IIIIZIZZ -0.01 IIIIZIZX -0.43 IIIIZZII +0.04 IIIIZZIZ -0.01 IIIIZZIX -0.06 IIIIZZZI +0.05 IIIIZZZZ +0.01 IIIIZZZX +0.00 IIIIXXXI +0.05 IIIIXXXZ -0.06 IIIIXXXX -0.01 IIIIXXYY -0.01 IIIIXYXY +0.01 IIIIXYYI +0.01 IIIIXYYZ -0.01 IIIIXYYX +0.06 IIIIYXXY -0.05 IIIIYXYI -0.00 IIIIYXYZ -0.01 IIIIYXYX +0.01 IIIIYYXI +0.01 IIIIYYXZ -0.01 IIIIYYXX +0.01 IIIIYYYY +0.24 IIIZIIII -0.09 IIIZIIIZ -0.03 IIIZIIIX -0.07 IIIZIIZZ +0.03 IIIZIIZX -0.05 IIIZIZZI +0.07 IIIZZIII -0.05 IIIZZZII -0.02 IIIZXXXZ +0.01 IIIZXXXX -0.01 IIIZYXXY +0.02 IIIZYXYI +0.08 IIIXIIII +0.03 IIIXIIIZ +0.03 IIIXIIIX +0.01 IIIXIIZZ -0.03 IIIXIIZX -0.02 IIIXIZZI +0.00 IIIXZIII -0.02 IIIXZZII +0.02 IIIXXXXZ -0.02 IIIXXXXX +0.02 IIIXYXXY -0.02 IIIXYXYI -0.04 IIZIIIII -0.31 IIZZIIII +0.07 IIZZIIIZ +0.01 IIZZ

In [145]:
circ = pc.circuit.Circuit(300)

In [146]:
for k in range(100):
    in1 = np.random.randint(0,300)
    in2 = np.random.randint(0,300)
    while in2==in1:
        in2 = np.random.randint(0,300)
    circ.gate(in1,in2)
    circ.measure(in1,in2)

In [147]:
state = pc.stabilizer.ghz_state(300)

In [148]:
%timeit circ.forward(state)

38.5 ms ± 550 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [149]:
circ.backward(state)

ValueError: Post-selection result is not possible, they are orthogonal states.

In [150]:
def binary_sum(x):
    # Summing in pyclifford seems to take quadratic time. More efficient to do a recursive sum over halves of the list.
    if len(x) < 30:
        return sum(x)
    else:
        return binary_sum(x[::2]) + binary_sum(x[1::2])

In [161]:
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper, BravyiKitaevMapper
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.algorithms import GroundStateEigensolver
from pyscf import scf, gto, fci
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)

In [162]:
def calculate_hamiltonian(geometry, use_pyscf=False, multiplicity=1, freeze=True, mapper=ParityMapper()):
    molecule = Molecule(geometry=geometry, charge=0, multiplicity=multiplicity)

    driver = ElectronicStructureMoleculeDriver(
        molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
    )

    es_problem = ElectronicStructureProblem(driver, transformers=[FreezeCoreTransformer()] if freeze else [])
    
    qubit_converter = QubitConverter(mapper, two_qubit_reduction=True, z2symmetry_reduction='auto')
    second_q_op = es_problem.second_q_ops()
    qubit_op = qubit_converter.convert(second_q_op['ElectronicEnergy'], num_particles=es_problem.num_particles).to_pauli_op().oplist
    shift = 0 # shift includes nuclear repulsion, core freezing, and a constant term (removing the identity). 
    nuclear_repuls = es_problem.grouped_property_transformed.get_property("ElectronicEnergy").nuclear_repulsion_energy
    shift += nuclear_repuls
    if freeze:
        core_shift = es_problem.grouped_property_transformed.get_property("ElectronicEnergy")._shift['FreezeCoreTransformer']
        shift += core_shift
    
    print('Done calculating qubit_op.')
    is_identity = lambda x: str(x.primitive).count('I') == len(str(x.primitive))
    const_shift = sum([x.coeff for x in qubit_op if is_identity(x)])
    shift += const_shift
    pyc_hamiltonian = binary_sum([x.coeff * pc.paulialg.pauli(str(x.primitive)) for x in qubit_op if not is_identity(x)])
    print('Done calculating pyclifford Hamiltonian.')
    if pyc_hamiltonian.N >= 14 or use_pyscf:
        print("Using pyscf ground state estimate")
        mol_s = '; '.join([x + ' ' + ' '.join(map(str, y)) for x,y in geometry])
        mol = gto.M(atom = mol_s, basis = 'sto3g', spin=multiplicity-1)
        rhf = scf.RHF(mol)
        E0 = rhf.kernel()
        return pyc_hamiltonian, np.real(E0),shift
    else:
        print("Using exact ground state")
        numpy_solver = NumPyMinimumEigensolver()
        calc = GroundStateEigensolver(qubit_converter, numpy_solver)
        res = calc.solve(es_problem)
        return pyc_hamiltonian, np.real(min(res.total_energies) ),shift

In [164]:
bond_len = 3.8
molecule = [["H", [0, 0.0, 0]], ["Li", [bond_len, 0, 0]]]
pyc_h, E0, shift = calculate_hamiltonian(molecule, use_pyscf=False,multiplicity=1, mapper=ParityMapper())
print('n_qubits:', pyc_h.N, 'E0', E0, 'n_terms', len(pyc_h))

Done calculating qubit_op.
Done calculating pyclifford Hamiltonian.
Using exact ground state
n_qubits: 8 E0 -7.7849965239549315 n_terms 275


In [165]:
pyc_h

-0.24 IIIIIIIZ +0.08 IIIIIIIX +0.04 IIIIIIZI -0.31 IIIIIIZZ -0.08 IIIIIIZX -0.06 IIIIIZII +0.05 IIIIIZIZ +0.01 IIIIIZIX -0.43 IIIIIZZI +0.04 IIIIIZZZ -0.01 IIIIIZZX +0.41 IIIIZIII -0.05 IIIIZIIZ +0.01 IIIIZIIX +0.07 IIIIZIZI -0.05 IIIIZIZZ -0.01 IIIIZIZX -0.43 IIIIZZII +0.04 IIIIZZIZ -0.01 IIIIZZIX -0.06 IIIIZZZI +0.05 IIIIZZZZ +0.01 IIIIZZZX +0.00 IIIIXXXI +0.05 IIIIXXXZ -0.06 IIIIXXXX -0.01 IIIIXXYY -0.01 IIIIXYXY +0.01 IIIIXYYI +0.01 IIIIXYYZ -0.01 IIIIXYYX +0.06 IIIIYXXY -0.05 IIIIYXYI -0.00 IIIIYXYZ -0.01 IIIIYXYX +0.01 IIIIYYXI +0.01 IIIIYYXZ -0.01 IIIIYYXX +0.01 IIIIYYYY +0.24 IIIZIIII -0.09 IIIZIIIZ -0.03 IIIZIIIX -0.07 IIIZIIZZ +0.03 IIIZIIZX -0.05 IIIZIZZI +0.07 IIIZZIII -0.05 IIIZZZII -0.02 IIIZXXXZ +0.01 IIIZXXXX -0.01 IIIZYXXY +0.02 IIIZYXYI +0.08 IIIXIIII +0.03 IIIXIIIZ +0.03 IIIXIIIX +0.01 IIIXIIZZ -0.03 IIIXIIZX -0.02 IIIXIZZI +0.00 IIIXZIII -0.02 IIIXZZII +0.02 IIIXXXXZ -0.02 IIIXXXXX +0.02 IIIXYXXY -0.02 IIIXYXYI -0.04 IIZIIIII -0.31 IIZZIIII +0.07 IIZZIIIZ +0.01 IIZZ